# Сербские желехные дороги

**Задача:** объединить файлы  Excel по условиям, разбить на категории(материалы, работы и оборудование) и сохранить в книгу Excel

In [39]:
import pandas as pd
import glob
import os
import re
import numpy as np
pd.options.mode.chained_assignment = None

In [40]:
#смотрим директорию 
os.getcwd()

'/Users/denisslobodyanyuk/Downloads/OneDrive-2022-06-23/ИДП 2'

####  Функция для кондиционирования и электроснабжения

In [41]:
# на вход получает список путей к файлам, условие для аждой категории, ИДП 1 или ИДП 2
def get_total_data_per_condition(xl_files_by_idp, condition, category):
   

    result_frame = pd.DataFrame()
    
    
    #цикл для создания одной таблицы
    for file in xl_files_by_idp:
        
        
        #проверяем на условие
        if condition in file:
            
            #блоки из определённых файлов
            #if file == '4.2.5_НЭ_Ресник Лапово_ИДП1.xls':
               
              #  data = pd.read_excel(file)
              #  data = data.loc[11:240]
                
            #elif file == '4.2.7_НЭ_Ниш_ИДП1.xls':
                
             #   data = pd.read_excel(file)
              #  data = pd.concat([data.loc[296:388], data.loc[392:485]])
            
            #elif file == '4.2.2_КТПМ_Лапово Ниш_ИДП2.xls':
                
             #   data = pd.read_excel(file)
              #  data = pd.concat([data.loc[3514:3727], data.loc[3731:3935], data.loc[3938:4116]])
                
            #else:
        
            data = pd.read_excel(file).drop(range(9), axis=0).reset_index(drop=True)
            
            
            #проверяем количество столбцов
            if len(data.columns) == 6:
                data['НИИАС'] = data['НИИАС'].fillna('unknown')
           
            elif len(data.columns) == 9:
                data = data.drop(['НИИАС', 'Unnamed: 7', 'Unnamed: 8'], axis=1)
            
            else:
                data = data.drop(['НИИАС', 'Unnamed: 7'], axis=1)

                
            #переименуем столбцы
            data.columns = ['description_rus', 'description_ser', 'measure', 'amount', 'price_per_item', 'total']

            
            #переведем столбцы в строки
            data['description_rus'] = data['description_rus'].astype('str')
            data['description_ser'] = data['description_ser'].astype('str')
            
            #пропуски заменим на 0
            data = data.fillna(0)

            #добавим столбец с категорией ИДП 1/ИДП 2
            data['category'] = category

            #соединим таблицы
            result_frame = pd.concat([result_frame, data])
            
    
    return result_frame

#### Функция для ВОЛС

In [42]:
def get_total_data_per_condition_vols(xl_files_by_idp, condition, category):
   

    result_frame = pd.DataFrame()
    
    #цикл для создания одной таблицы
    for file in xl_files_by_idp:
        
        
        if condition in file:
        
        #удалим первые ненужные строки
            data = pd.read_excel(file).drop(range(9), axis=0).drop('НИИАС', axis=1).reset_index(drop=True)
            

                
            #переименуем столбцы
            data.columns = ['description_rus', 'measure', 'amount', 'price_per_item', 'total']

            
            data['description_rus'] = data['description_rus'].astype('str')
            #data['description_ser'] = data['description_ser'].astype('str')
            data = data.fillna(0)

            #добавим столбец с категорией ИДП 1/ИДП 2
            data['category'] = category

            #соединим таблицы
            result_frame = pd.concat([result_frame, data])
    
    return result_frame

####  Функция для ТСОБ

In [43]:
def get_total_data_per_condition_tsob(xl_files_by_idp, condition, category):
   

    result_frame = pd.DataFrame()
    
    #цикл для создания одной таблицы
    for file in xl_files_by_idp:
        
        
        if condition in file:
        
        #удалим первые ненужные строки
            data = pd.read_excel(file).drop(range(9), axis=0).reset_index(drop=True)
            
            
            if len(data.columns) == 6:
                data = data.drop('Unnamed: 2', axis=1)
            elif len(data.columns) == 9:
                data = data.drop(['НИИАС', 'Unnamed: 2', 'Unnamed: 7', 'Unnamed: 8'], axis=1)
            elif len(data.columns) == 10:
                data = data.drop(['НИИАС', 'Unnamed: 2', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9'], axis=1)
            elif len(data.columns) == 11:
                data = data.drop(['НИИАС', 'Unnamed: 2', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10'], axis=1)
            elif len(data.columns) == 12:
                data = data.drop(['НИИАС', 'Unnamed: 2', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11'], axis=1)
            elif len(data.columns) == 7:
                data = data.drop(['НИИАС','Unnamed: 2'], axis=1)
            else:
                data = data.drop(['НИИАС', 'Unnamed: 2', 'Unnamed: 7'], axis=1)

                
            #переименуем столбцы
            data.columns = ['description_ser', 'measure', 'amount', 'price_per_item', 'total']

            
            data['description_ser'] = data['description_ser'].astype('str')
            
            data = data.fillna(0)

            #добавим столбец с категорией ИДП 1/ИДП 2
            data['category'] = category

            #соединим таблицы
            result_frame = pd.concat([result_frame, data])
            
    
    return result_frame

####  Функция для категоризации материалов и работ

In [44]:
def materials_or_works(data):
    
    #столбец для категорий
    data['name'] = ''


    for x in data.index:                

        work = re.search('работы$', data.loc[x]['description_rus'])
        equipment = re.search('оборудование$', data.loc[x]['description_rus'])
        
        if  ('Материалы' in data.loc[x]['description_rus']
             or 'Материјали' in data.loc[x]['description_ser']
              or 'материјали' in data.loc[x]['description_ser']):         
            
             data['name'].loc[x:] = 'материалы'


        elif ('Оборудование' in data.loc[x]['description_rus']
              or 'опрема' in data.loc[x]['description_ser']
              or 'Опрема' in data.loc[x]['description_ser']
              or equipment != None):           
            
             data['name'].loc[x:] = 'оборудование'


        elif ('Работы' in data.loc[x]['description_rus']
              or work != None
              or 'радови' in data.loc[x]['description_rus']
              or 'радови' in data.loc[x]['description_ser']
              or 'Радови' in data.loc[x]['description_ser']):           
            
             data['name'].loc[x:] = 'работы'

#### Функция для категоризации работ, где только описание на сербском 

In [45]:
def materials_or_works_only_serbian(data):
    
    data['name'] = ''


    for x in data.index:                

        
        if  ( 'Материјали' in data.loc[x]['description_ser']
              or 'материјали' in data.loc[x]['description_ser']):         
            
             data['name'].loc[x:] = 'материалы'


        elif ( 'опрема' in data.loc[x]['description_ser']
              or 'Опрема' in data.loc[x]['description_ser']):           
            
             data['name'].loc[x:] = 'оборудование'


        elif (  'радови' in data.loc[x]['description_ser']
              or 'Радови' in data.loc[x]['description_ser']):           
            
             data['name'].loc[x:] = 'работы'

In [46]:
def materials_or_works_only_russian(data):
    
    data['name'] = ''


    for x in data.index:                

        work = re.search('работы$', data.loc[x]['description_rus'])
        
        if  ( 'Материалы' in data.loc[x]['description_rus']
              or 'материалы' in data.loc[x]['description_rus']):         
            
             data['name'].loc[x:] = 'материалы'


        elif ( 'Оборудование' in data.loc[x]['description_rus']
              or 'Оборудование' in data.loc[x]['description_rus']):           
            
             data['name'].loc[x:] = 'оборудование'


        elif ( 'Работы' in data.loc[x]['description_rus']
              or work != None):           
            
             data['name'].loc[x:] = 'работы'

## 1. Работа с файлами " ИДП 1"

In [47]:
#меняем на нужную папку
os.chdir('/Users/denisslobodyanyuk/Downloads/OneDrive-2022-06-23/ИДП 1')


#сохраним пути ко всем файлам
xl_files_idp_1 = glob.glob('*.xls')

 #### 1.1 Файлы 'НЕТЯГОВОЕ ЭЛЕКТРОСНАБЖЕНИЕ ИДП1'

In [48]:
idp1_electricity = get_total_data_per_condition(xl_files_idp_1, 'НЭ', 'ИДП 1').reset_index(drop=True)

materials_or_works(idp1_electricity)
                    
                    
idp1_electricity.head()        

,description_rus,description_ser,measure,amount,price_per_item,total,category,name
0,Материалы и оборудование,Материјали и опрема,0,0.0,0,0.0,ИДП 1,материалы
1,Комплектная трансформаторная подстанция мачтов...,Стубна трансформаторска станица комплетне фабр...,0,0.0,0,0.0,ИДП 1,материалы
2,Оборудование,Опрема,0,0.0,0,0.0,ИДП 1,оборудование
3,Трансформатор силовой трехфазный/однофазный 25...,"Трансформатор снаге трофазни/монофазни 25/0,4 ...",шт.,1.0,2000,2000.0,ИДП 1,оборудование
4,Предохранитель высоковольтный,Осигурач високог напона,шт.,1.0,99,99.0,ИДП 1,оборудование


####  1.2 Файлы  'ТСОБ ИДП 1'

In [49]:
idp1_tsob = get_total_data_per_condition_tsob(xl_files_idp_1, 'ТСОБ', 'ИДП 1').drop(0, axis=0).reset_index(drop=True)

materials_or_works_only_serbian(idp1_tsob)


idp1_tsob.head()

,description_ser,measure,amount,price_per_item,total,category,name
0,"Опрема, материјали",0,0,0.00,0.00,ИДП 1,материалы
1,4Mp спољна куполна IP-камера са EXIR-осветљење...,ком.,3,639.00,1917.00,ИДП 1,материалы
2,4Mp спољна цилиндрична IP-камера са EXIR-освет...,ком.,13,640.00,8320.00,ИДП 1,материалы
3,Комутатор PSW-2G6F+ (6х10/100Base-Tx RJ-45 PoE...,ком.,3,572.52,1717.56,ИДП 1,материалы
4,Трансивер (примопредајник) 1000BASE-LX/LH SFP ...,ком.,7,991.43,6940.01,ИДП 1,материалы


####  1.3 Файлы 'КОНДИЦИОНИРОВАНИЕ ИДП 1'

In [50]:
idp1_conditioning = get_total_data_per_condition(xl_files_idp_1, 'Кондиционирование', 'ИДП 1').drop(0, axis=0).reset_index(drop=True)

materials_or_works(idp1_conditioning)



idp1_conditioning.head()

,description_rus,description_ser,measure,amount,price_per_item,total,category,name
0,Оборудование,Опрема,0,0.0,0.00,0.00,ИДП 1,оборудование
1,Комплект кондиционера SRK60ZR-S / SRC60ZR-S,Комплетан клима-уређај SRK60ZR-S / SRC60ZR-S,шт.,4.0,1974.50,7898.00,ИДП 1,оборудование
2,Пульт управления проводной,Жичани даљински управљач,шт.,2.0,164.62,329.24,ИДП 1,оборудование
3,Нагреватель дренажной трубки,Грејач дренажне цеви,шт.,4.0,25.56,102.24,ИДП 1,оборудование
4,Согласователь работы кондиционеров,Контролер рада клима-уређаја,шт.,1.0,465.50,465.50,ИДП 1,оборудование


#### 1.4 Файлы  'ВОЛС ИДП 1'

In [51]:
idp1_vols = get_total_data_per_condition_vols(xl_files_idp_1, 'ЛОКАЛЬНЫЕ', 'ИДП 1').drop(0, axis=0).reset_index(drop=True)

materials_or_works_only_russian(idp1_vols)



idp1_vols.head()

,description_rus,measure,amount,price_per_item,total,category,name
0,Оборудование,0,0.0,0.00,0.00,ИДП 1,оборудование
1,"Шкаф напольный 19-дюймовый, 42U, 2055х600х800 ...",шт.,1.0,785.00,785.00,ИДП 1,оборудование
2,"Шкаф настенный 19-дюймовый, 6U, 370х600х600 мм...",шт.,1.0,234.43,234.43,ИДП 1,оборудование
3,Материалы,0,0.0,0.00,0.00,ИДП 1,материалы
4,Комплектующие оптических шкафов,0,0.0,0.00,0.00,ИДП 1,материалы


### 2. Работа с файлами "ИДП 2"

In [52]:
#теперь делаем тоже самое для ИДП 2
os.chdir('/Users/denisslobodyanyuk/Downloads/OneDrive-2022-06-23/ИДП 2')


xl_files_idp_2 = glob.glob('*.xls')
xl_files_idp_2 = xl_files_idp_2 + glob.glob('*.xlsx')

#### 2.1  Файлы 'Электроснабжение ИДП 2'

In [53]:
idp2_electricity = get_total_data_per_condition(xl_files_idp_2, 'ТП', 'ИДП 2').reset_index(drop=True) 

materials_or_works(idp2_electricity)

        
idp2_electricity.head()

,description_rus,description_ser,measure,amount,price_per_item,total,category,name
0,unknown,Опрема,0,0.0,0,0.0,ИДП 2,оборудование
1,unknown,Енергетски трансформатор трофазни/једнофазни ...,ком.,1.0,2000,2000.0,ИДП 2,оборудование
2,unknown,Високонапонски осигурач,ком.,1.0,99,99.0,ИДП 2,оборудование
3,unknown,Одводник пренапона,ком.,1.0,199,199.0,ИДП 2,оборудование
4,unknown,Растављач,ком.,1.0,633,633.0,ИДП 2,оборудование


#### 2.2 Файлы 'ТСОБ ИДП 2'

In [54]:
idp2_tsob = get_total_data_per_condition_tsob(xl_files_idp_2, '06.0', 'ИДП 2').drop(0, axis=0).reset_index(drop=True)

materials_or_works_only_serbian(idp2_tsob)


idp2_tsob.head()

,description_ser,measure,amount,price_per_item,total,category,name
0,"Опрема, материјали",0,0.0,0.00,0.0,ИДП 2,материалы
1,4Mp спољна куполна IP-камера са EXIR-осветљење...,ком.,1.0,639.00,639.0,ИДП 2,материалы
2,4Mp спољна цилиндрична IP-камера са EXIR-освет...,ком.,4.0,640.00,2560.0,ИДП 2,материалы
3,Комутатор PSW-2G6F+ (6х10/100Base-Tx RJ-45 Po...,ком.,1.0,572.52,572.52,ИДП 2,материалы
4,Трансивер (примопредајник) 1000BASE-LX/LH SFP ...,ком.,8.0,991.43,7931.44,ИДП 2,материалы


#### 2.3  Файлы  'Кондиционирование ИДП 2'

In [55]:
idp2_conditioning = get_total_data_per_condition(xl_files_idp_2, 'КОНДИЦИОНИРОВАНИЕ', 'ИДП 2').drop(0, axis=0).reset_index(drop=True)

materials_or_works(idp2_conditioning)

idp2_conditioning.head()

,description_rus,description_ser,measure,amount,price_per_item,total,category,name
0,Оборудование,Опрема,0,0,0.00,0.00,ИДП 2,оборудование
1,Комплект кондиционера SRK60ZSX-S / SRC60ZSX-S,Комплетан клима-уређај SRK60ZSX-S / SRC60ZSX-S,шт.,2,1974.50,3949.00,ИДП 2,оборудование
2,Пульт управления проводной,Жичани даљински управљач,шт.,1,164.62,164.62,ИДП 2,оборудование
3,Нагреватель дренажной трубки,Грејач дренажне цеви,шт.,2,25.56,51.12,ИДП 2,оборудование
4,Согласователь работы кондиционеров,Контролер рада клима-уређаја,шт.,1,465.50,465.50,ИДП 2,оборудование


#### 2.4 Файлы 'Волс ИДП 2'

In [56]:
idp2_vols = get_total_data_per_condition_vols(xl_files_idp_2, 'ЛОКАЛЬНЫЕ', 'ИДП 2').drop(0, axis=0).reset_index(drop=True)

materials_or_works_only_russian(idp2_vols)



idp2_vols.head()

/Users/denisslobodyanyuk/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,description_rus,measure,amount,price_per_item,total,category,name
0,Оборудование,0,0.0,0.00,0.00,ИДП 2,оборудование
1,"Шкаф напольный 19-дюймовый, 42U, 2055х600х800 ...",шт.,1.0,785.00,785.00,ИДП 2,оборудование
2,"Шкаф настенный 19-дюймовый, 6U, 370х600х600 мм...",шт.,1.0,234.43,234.43,ИДП 2,оборудование
3,Материалы,0,0.0,0.00,0.00,ИДП 2,материалы
4,Комплектующие оптических шкафов,0,0.0,0.00,0.00,ИДП 2,материалы


###  3 Общие таблицы по двум ИДП

### 3.1 Электроснабжение 

In [57]:
total_electricity = pd.concat([idp1_electricity, idp2_electricity]).drop(range(2), axis=0).reset_index(drop=True)

#### 3.1.1 Сводная таблица для материалов и оборудования

In [58]:
#выбираем нужные строи и удалим таие строки ка "Всего работ"
electricity_materials = total_electricity.query('name == ["материалы", "оборудование"] & price_per_item != 0').reset_index(drop=True)

#там, где nan, там нет описания 
electricity_materials.loc[(electricity_materials.description_ser == 'nan'), 'description_ser'] = 'no description'


electricity_materials_pivot = electricity_materials.pivot_table(                        
    index = ['name', 'category', 'description_rus', 'description_ser', 'measure', 'price_per_item'],
    values = ['amount', 'total'],
    aggfunc ='sum',
    margins = True,
    margins_name = 'TOTAL').reset_index()



electricity_materials_pivot.tail()

,name,category,description_rus,description_ser,measure,price_per_item,amount,total
776,оборудование,ИДП 2,Щит распределительный навесной,Разводна табла са шаркама,шт.,7.6,141.0,1071.60
777,оборудование,ИДП 2,Щит с монтажной панелью ЩМП 600x400x210,Табла са монтажним панелом ТМП 600x400x210,шт.,71.5,2.0,143.00
778,оборудование,ИДП 2,Щит с монтажной панелью ЩМП-03 350x300x155,Табла са монтажним панелом ТМП-03 350x300x155,шт.,27.7,79.0,2188.30
779,оборудование,ИДП 2,Щит с монтажной панелью ЩМП-04 400x300x155,Табла са монтажним панелом ТМП-04 400x300x155,шт.,32.3,80.0,2584.00
780,TOTAL,,,,,,346013.8,2282781.79


#### 3.1.2 Сводная таблица для работ

In [59]:
electricity_works = total_electricity.query('name == "работы" & price_per_item != 0').reset_index(drop=True)

electricity_works.loc[(electricity_works.description_ser == 'nan'), 'description_ser'] = 'no description'

electricity_works_pivot = electricity_works.pivot_table(                        
    index = ['category', 'description_rus', 'description_ser', 'measure', 'price_per_item'],
    values = ['amount', 'total'],
    aggfunc ='sum',
    margins = True,
    margins_name = 'TOTAL').reset_index()



electricity_works_pivot.tail()

,category,description_rus,description_ser,measure,price_per_item,amount,total
323,ИДП 2,"Установка шин «N», «РЕ» в шкаф","Монтажа шина «N», «РЕ» у орман",шт.,4.84,2.0,9.68
324,ИДП 2,Установка щита ЩАО с подключением,Уградња табле ХО са прикључком,шт.,43.81,79.0,3460.99
325,ИДП 2,Установка щита ЩР1 с подключением,Уградња табле РТ1 са прикључком,шт.,43.81,71.0,3110.51
326,ИДП 2,Установка щита ЩР3 с подключением к потребителям,Уградња табле РТ3 са прикључком према потрошачима,шт.,43.81,22.0,963.82
327,TOTAL,,,,,130672.0,889493.31


#### 

### 3.2 ТСОБ

In [60]:
total_tsob = pd.concat([idp1_tsob, idp2_tsob]).reset_index(drop=True)

#приведем к численному значению
total_tsob.loc[(total_tsob.amount == '2 516'), 'amount'] = 2516
total_tsob.loc[(total_tsob.amount == '1 024'), 'amount'] = 1024


#изменим тип данных на нужный
total_tsob['amount'] = total_tsob['amount'].astype('float64')
total_tsob['total'] = total_tsob['total'].astype('float64')
total_tsob.query('name == "работы"').head(40)

,description_ser,measure,amount,price_per_item,total,category,name
68,Монтажни радови,0,0.000,0.00,0.00,ИДП 1,работы
69,Пробијање у зидовима од опеке округлих отвора ...,100 ком.,0.020,3754.99,75.10,ИДП 1,работы
70,Постављање металних стубова висине до 4 м са п...,100 стубова,0.060,7510.00,450.60,ИДП 1,работы
71,Постављање монтажних уложака за спајање тежине...,т,0.660,1289.20,850.87,ИДП 1,работы
72,Уградња цевовода од полиетиленских цеви са бро...,1 км цевовода,0.314,1423.14,446.87,ИДП 1,работы
73,Постављање азбестноцементних заштитних цеви пр...,100 м цеви,0.150,234.06,35.11,ИДП 1,работы
74,Пробијање у зидовима од опеке округлих отвора ...,100 ком.,0.040,3754.99,150.20,ИДП 1,работы
75,Пробијање у зидовима од опеке округлих отвора ...,100 ком.,0.040,3754.99,150.20,ИДП 1,работы
76,Херметизација пролаза код увода кабла у експло...,1 пролаз кабла,2.000,27.54,55.08,ИДП 1,работы
77,Херметизација пролаза код увода кабла у експло...,1 пролаз кабла,4.000,27.54,110.16,ИДП 1,работы


#### 3.2.1 Сводная таблица для материалов и оборудования

In [61]:
tsob_materials = total_tsob.query('name == ["материалы", "оборудование"] & price_per_item != 0').reset_index(drop=True)


tsob_materials_pivot = tsob_materials.pivot_table(                        
    index = ['category', 'description_ser', 'measure', 'price_per_item'],
    values = ['amount', 'total'],
    aggfunc ='sum',
    margins = True,
    margins_name = 'TOTAL').reset_index()



tsob_materials_pivot.tail()

,category,description_ser,measure,price_per_item,amount,total
636,ИДП 2,Читач кода проксимити картица (формат картица ...,ком.,90.68,508.000,46065.44
637,ИДП 2,Шина за уземљење вертикална 1000 мм / 200 А ПЗ...,ком.,31.09,6.000,186.54
638,ИДП 2,Шина за уземљење вертикална 1000 мм / 200 А ПЗ...,ком.,31.09,4.000,124.36
639,ИДП 2,"Шина за уземљење хоризонтална/вертикална 19"" 5...",ком.,17.6,10.000,176.00
640,TOTAL,,,,539885.792,15000835.31


#### 3.2.2 Сводная таблица для работ

In [62]:
tsob_works = total_tsob.query('name == "работы" & price_per_item != 0').reset_index(drop=True)

tsob_works.loc[(tsob_works.description_ser == 'nan'), 'description_ser'] = 'no description'

tsob_works_pivot = tsob_works.pivot_table(                        
    index = ['category', 'description_ser', 'measure', 'price_per_item'],
    values = ['amount', 'total'],
    aggfunc ='sum',
    margins = True,
    margins_name = 'TOTAL').reset_index()



tsob_works_pivot.head()

,category,description_ser,measure,price_per_item,amount,total
0,ИДП 1,4Mp спољна куполна IP-камера са EXIR-осветљење...,ком.,50.07,24.0,1201.68
1,ИДП 1,"Аутоматски прекидач једно-, дво-, трополни кој...",ком.,20.02,1336.0,26746.72
2,ИДП 1,"Аутоматски прекидач једно-, дво-, трополни кој...",ком.,13.77,6.0,82.62
3,ИДП 1,"Аутоматски прекидач једно-, дво-, трополни кој...",ком.,13.77,23.0,316.71
4,ИДП 1,"Аутоматски прекидач једно-, дво-, трополни кој...",ком.,15.02,134.0,2012.68


### 3.3  Кондиционирование 

In [63]:
total_conditioning = pd.concat([idp1_conditioning, idp2_conditioning]).reset_index(drop=True)

#### 3.3.1 Сводная таблица для материалов и оборудования

In [64]:
# выбираем материалы и оборудование
conditioning_materials = total_conditioning.query('name == ["материалы", "оборудование"] & price_per_item != 0').reset_index(drop=True)

# сводная таблица
conditioning_materials_pivot = conditioning_materials.pivot_table(                        
    index = ['name', 'category', 'description_rus', 'description_ser', 'measure', 'price_per_item'],
    values = ['amount', 'total'],
    aggfunc ='sum',
    margins = True,
    margins_name = 'TOTAL').reset_index()



conditioning_materials_pivot.tail()

,name,category,description_rus,description_ser,measure,price_per_item,amount,total
44,оборудование,ИДП 2,Комплект кондиционера SRK80ZR-S / SRC80ZR-S,Комплетан клима-уређај SRK80ZR-S / SRC80ZR-S,шт.,1974.5,20.0,39490.00
45,оборудование,ИДП 2,Нагреватель дренажной трубки,Грејач дренажне цеви,шт.,25.56,172.0,4396.32
46,оборудование,ИДП 2,Пульт управления проводной,Жичани даљински управљач,шт.,164.62,139.0,22882.18
47,оборудование,ИДП 2,Согласователь работы кондиционеров,Контролер рада клима-уређаја,шт.,465.5,74.0,34447.00
48,TOTAL,,,,,,21177.8,840185.08


#### 3.3.2 Сводная таблица для работ

In [65]:
#выбираем работы
conditioning_works = total_conditioning.query('name == "работы" & price_per_item != 0').reset_index(drop=True)


#сводная таблица
conditioning_works_pivot = conditioning_works.pivot_table(                        
    index = ['category', 'description_rus', 'description_ser', 'measure', 'price_per_item'],
    values = ['amount', 'total'],
    aggfunc ='sum',
    margins = True,
    margins_name = 'TOTAL').reset_index()



conditioning_works_pivot.tail()

,category,description_rus,description_ser,measure,price_per_item,amount,total
36,ИДП 2,Прокладка и монтаж дренажной трубки 16мм в коробе,Полагање и монтажа дренажне цеви 16мм у каналици,м,3.0,1234.0,3702.00
37,ИДП 2,Прокладка и монтаж кабеля NKL в коробе,Полагање и монтажа кабла NKL у каналици,м,1.5,1067.0,1600.50
38,ИДП 2,Прокладка и монтаж трубопроводов 15.9мм х 0.09...,Полагање и монтажа цевовода 15.9мм х 0.09 мм ...,м,7.45,1220.0,9089.00
39,ИДП 2,Прокладка и монтаж трубопроводов 9.5мм х 0.01 ...,Полагање и монтажа цевовода 9.5мм х 0.01 мм у ...,м,3.73,1320.0,4923.60
40,TOTAL,,,,,12406.8,198510.85


### 3.4  ВОЛС 

In [66]:
total_vols = pd.concat([idp1_vols, idp2_vols]).reset_index(drop=True)

#### 3.4.1 сводная таблица для материалов и оборудования 

In [67]:
# выбираем материалы и оборудование
vols_materials = total_vols.query('name == ["материалы", "оборудование"] & price_per_item != 0').reset_index(drop=True)

# сводная таблица
vols_materials_pivot = vols_materials.pivot_table(                        
    index = ['name', 'category', 'description_rus', 'measure', 'price_per_item'],
    values = ['amount', 'total'],
    aggfunc ='sum',
    margins = True,
    margins_name = 'TOTAL').reset_index()



vols_materials_pivot.head()

,name,category,description_rus,measure,price_per_item,amount,total
0,материалы,ИДП 1,Hyperline CM-1U-PL Кабельный организатор с пла...,шт.,11.76,160.0,1881.60
1,материалы,ИДП 1,NKL-F-012A1J-00B-BK Кабель волоконно-оптически...,м,2.2,24245.5,53340.10
2,материалы,ИДП 1,NTSS-RFOB-1U-12-E2000/A-9-SP2*1.5G1A Кросс 1...,шт.,100.4,220.0,22088.00
3,материалы,ИДП 1,"Кабель заземления, желто-зеленый",м,0.79,1650.0,1303.50
4,материалы,ИДП 1,Кабель-канал 15х10,м,0.51,115.0,58.65


#### 3.4.2 Сводная таблица для работ

In [68]:
#выбираем работы
vols_works = total_vols.query('name == "работы" & price_per_item != 0').reset_index(drop=True)


#сводная таблица
vols_works_pivot = vols_works.pivot_table(                        
    index = ['category', 'description_rus', 'measure', 'price_per_item'],
    values = ['amount', 'total'],
    aggfunc ='sum',
    margins = True,
    margins_name = 'TOTAL').reset_index()



vols_works_pivot.head()

,category,description_rus,measure,price_per_item,amount,total
0,ИДП 1,Вывоз грунта на полигон ТБО,м3,11.0,1283.2,14115.20
1,ИДП 1,Засыпка траншеи песком,м3,6.8,1283.2,8725.75
2,ИДП 1,Монтаж и сборка оптического кросса,шт.,37.44,220.0,8236.80
3,ИДП 1,"Монтаж шкафа телекоммуникационного напольного,...",шт.,84.2,55.0,4631.00
4,ИДП 1,"Монтаж шкафа телекоммуникационного напольного,6 U",шт.,28.44,55.0,1564.20


## 4. Сохранение в EXCEL

### 4.1 Кондиционирование

In [69]:
# путь в файлу xls
file_1 = '/Users/denisslobodyanyuk/Desktop/Сметы ИДП 1 и ИДП 2/сводные таблицы/общая кондиционирование .xlsx'

table_1 = pd.ExcelWriter(file_1, engine='xlsxwriter')

conditioning_materials_pivot.to_excel(table_1, 'материалы')
conditioning_works_pivot.to_excel(table_1, 'работы')

#ширина столбцов
sheet = table_1.sheets['материалы']
sheet.set_column('A:B', 10)
sheet.set_column('C:D', 50)
sheet.set_column('E:H', 15)

sheet = table_1.sheets['работы']
sheet.set_column('A:A', 10)
sheet.set_column('B:C', 70)
sheet.set_column('D:G', 15)

table_1.save()

###  4.2 ТСОБ

In [70]:
file_2 = '/Users/denisslobodyanyuk/Desktop/Сметы ИДП 1 и ИДП 2/сводные таблицы/общая ТСОБ .xlsx'

table_2 = pd.ExcelWriter(file_2, engine='xlsxwriter')

tsob_materials_pivot.to_excel(table_2, 'материалы')
tsob_works_pivot.to_excel(table_2, 'работы')

sheet = table_2.sheets['материалы']
#sheet.set_column('A:B', 10)
#sheet.set_column('C:C', 65)
#sheet.set_column('D:G', 15)

table_2.save()

## 4.3 Элетроснабжение

In [71]:
# путь в файлу xls
file_3 = '/Users/denisslobodyanyuk/Desktop/Сметы ИДП 1 и ИДП 2/сводные таблицы/общая электроснабжение .xlsx'

table_3 = pd.ExcelWriter(file_3, engine='xlsxwriter')

electricity_materials_pivot.to_excel(table_3, 'материалы')
electricity_works_pivot.to_excel(table_3, 'работы')

#ширина столбцов
sheet = table_3.sheets['материалы']
sheet.set_column('A:B', 10)
sheet.set_column('C:D', 50)
sheet.set_column('E:H', 15)

sheet = table_3.sheets['работы']
sheet.set_column('A:A', 10)
sheet.set_column('B:C', 70)
sheet.set_column('D:G', 15)

table_3.save()

## 4.4 Волс 

In [72]:
file_4 = '/Users/denisslobodyanyuk/Desktop/Сметы ИДП 1 и ИДП 2/сводные таблицы/общая ВОЛС .xlsx'

table_4 = pd.ExcelWriter(file_4, engine='xlsxwriter')

vols_materials_pivot.to_excel(table_4, 'материалы')
vols_works_pivot.to_excel(table_4, 'работы')

sheet = table_4.sheets['материалы']
#sheet.set_column('A:B', 10)
#sheet.set_column('C:C', 65)
#sheet.set_column('D:G', 15)

table_4.save()